# "2019 December One Day Holding Long Short Portfolio BackTest"
> "2019 December Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [1]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [2]:
#hide_input
df_testp=pd.read_csv('D:\\Downloads\\DeLaMachine\\EDGAR\\1dayreturnpred2019train60K.csv')

In [3]:
#hide
df_testp=df_testp.loc[df_testp.Date>=20191201].loc[df_testp.Date<20191232].sort_values(by='Date',ascending=True).copy()

**Long-Short Portfolio**

In [4]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [5]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500
0,2019-12-03,-1.0,-1.924002,1.019240,1.006324,1.005344
1,2019-12-03,1.0,-1.629895,1.002627,1.006324,1.005344
2,2019-12-05,-1.0,0.151976,1.001104,1.017041,1.018625
3,2019-12-05,1.0,22.662904,1.227983,1.017041,1.018625
4,2019-12-06,-1.0,-0.547945,1.234712,1.013824,1.014791
5,2019-12-06,1.0,5.859529,1.307060,1.013824,1.014791
6,2019-12-09,1.0,24.316007,1.624885,1.012712,1.013777
7,2019-12-10,1.0,-0.846267,1.611134,1.015657,1.014853
8,2019-12-12,-1.0,-7.205615,1.727226,1.024441,1.023000
9,2019-12-12,1.0,-4.509933,1.649329,1.024441,1.023000


In [7]:
#hide_input

a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='Portfolio (blue) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3  

alt.LayerChart(...)

**Long portfolio**

In [8]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500
0,2019-12-03,1.0,-1.629895,0.983701,1.006324,1.005344
1,2019-12-05,1.0,22.662904,1.206636,1.017041,1.018625
2,2019-12-06,1.0,5.859529,1.277339,1.013824,1.014791
3,2019-12-09,1.0,24.316007,1.587937,1.012712,1.013777
4,2019-12-10,1.0,-0.846267,1.574499,1.015657,1.014853
5,2019-12-12,1.0,-4.509933,1.503490,1.024441,1.023000
6,2019-12-13,1.0,-0.684215,1.493203,1.031763,1.026655
7,2019-12-16,1.0,-0.605028,1.484169,1.032109,1.027792
8,2019-12-17,1.0,-9.425638,1.344277,1.031663,1.026778
9,2019-12-18,1.0,-0.574332,1.336556,1.036263,1.031784


In [11]:
#hide_input

a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='Portfolio (blue) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3  

alt.LayerChart(...)

**Short Portfolio**

In [12]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500
0,2019-12-03,-1.0,-1.924002,1.019240,1.006324,1.005344
1,2019-12-05,-1.0,0.151976,1.017691,1.017041,1.018625
2,2019-12-06,-1.0,-0.547945,1.023267,1.013824,1.014791
3,2019-12-12,-1.0,-7.205615,1.097000,1.024441,1.023000
4,2019-12-16,-1.0,17.368188,0.906471,1.032109,1.027792
5,2019-12-19,-1.0,11.963650,0.798024,1.041388,1.034625
6,2019-12-20,-1.0,0.192678,0.796486,1.042290,1.038131
7,2019-12-23,-1.0,-5.119454,0.837262,1.042290,1.038131


In [15]:
#hide_input

a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='Portfolio (blue) vs Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_line(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3  

alt.LayerChart(...)